# Quality Assessment

## Purpose

This notebook teaches you how to assess data quality in voxel grids. You'll learn to evaluate completeness, signal quality, alignment accuracy, and overall data quality with interactive widgets.

## Learning Objectives

By the end of this notebook, you will:
- ✅ Assess data completeness and coverage
- ✅ Evaluate signal quality (SNR, uncertainty, confidence)
- ✅ Validate alignment accuracy
- ✅ Identify and fill gaps in data
- ✅ Make quality-based decisions

## Estimated Duration

60-90 minutes

---

## Overview

Quality assessment is essential for ensuring reliable analysis results. The AM-QADF framework provides comprehensive quality metrics:

- 📊 **Data Quality**: Completeness, coverage, consistency, accuracy, reliability
- 📈 **Signal Quality**: SNR, uncertainty, confidence scores
- ✅ **Alignment Accuracy**: Coordinate, temporal, and spatial accuracy
- 🔍 **Completeness**: Gap detection and filling strategies

Use the interactive widgets below to assess quality - no coding required!


In [1]:
# Setup: Import required libraries
import sys
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Add parent directory and src directory to path for imports
notebook_dir = Path().resolve()
project_root = notebook_dir.parent
src_dir = project_root / 'src'

# Add project root to path (for src.infrastructure imports)
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Add src directory to path (for am_qadf imports)
if str(src_dir) not in sys.path:
    sys.path.insert(0, str(src_dir))

# Core imports
import ipywidgets as widgets
from ipywidgets import (
    VBox, HBox, Accordion, Tab, Dropdown, RadioButtons, 
    Checkbox, Button, Output, Text, IntSlider, FloatSlider,
    Layout, Box, Label, FloatText, IntText
)
from IPython.display import display, Markdown, HTML, clear_output
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from typing import Optional, Tuple, Dict, Any, List

# Load environment variables from development.env
import os
env_file = project_root / 'development.env'
if env_file.exists():
    with open(env_file, 'r') as f:
        for line in f:
            line = line.strip()
            if line and not line.startswith('#') and '=' in line:
                key, value = line.split('=', 1)
                value = value.strip('"\'')
                os.environ[key] = value
    print("✅ Environment variables loaded from development.env")

# Try to import quality assessment classes
QUALITY_AVAILABLE = False
try:
    from am_qadf.quality.quality_assessment_client import QualityAssessmentClient
    from am_qadf.analytics.quality_assessment.data_quality import DataQualityAnalyzer, DataQualityMetrics
    QUALITY_AVAILABLE = True
    print("✅ Quality assessment classes available")
except ImportError as e:
    print(f"⚠️ Quality assessment classes not available: {e} - using demo mode")

# MongoDB connection setup
INFRASTRUCTURE_AVAILABLE = False
mongo_client = None
voxel_storage = None
stl_client = None

try:
    from src.infrastructure.config import MongoDBConfig
    from src.infrastructure.database import MongoDBClient
    from am_qadf.voxel_domain import VoxelGridStorage
    from am_qadf.query import STLModelClient
    
    # Initialize MongoDB connection
    config = MongoDBConfig.from_env()
    if not config.username:
        config.username = os.getenv('MONGO_ROOT_USERNAME', 'admin')
    if not config.password:
        config.password = os.getenv('MONGO_ROOT_PASSWORD', 'password')
    
    mongo_client = MongoDBClient(config=config)
    if mongo_client.is_connected():
        voxel_storage = VoxelGridStorage(mongo_client=mongo_client)
        stl_client = STLModelClient(mongo_client=mongo_client)
        INFRASTRUCTURE_AVAILABLE = True
        print(f"✅ Connected to MongoDB: {config.database}")
    else:
        print("⚠️ MongoDB connection failed")
except Exception as e:
    print(f"⚠️ MongoDB not available: {e} - using demo mode")

print("✅ Setup complete!")


✅ Environment variables loaded from development.env


✅ Quality assessment classes available


Failed to connect to MongoDB: localhost:27017: [Errno 111] Connection refused (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms), Timeout: 30.0s, Topology Description: <TopologyDescription id: 696011cad722e79b1b9091aa, topology_type: Unknown, servers: [<ServerDescription ('localhost', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('localhost:27017: [Errno 111] Connection refused (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms)')>]>


⚠️ MongoDB not available: localhost:27017: [Errno 111] Connection refused (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms), Timeout: 30.0s, Topology Description: <TopologyDescription id: 696011cad722e79b1b9091aa, topology_type: Unknown, servers: [<ServerDescription ('localhost', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('localhost:27017: [Errno 111] Connection refused (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms)')>]> - using demo mode
✅ Setup complete!


## Interactive Quality Assessment Interface

Use the widgets below to assess data quality. Select assessment type, configure metrics, and visualize quality results interactively!


In [2]:
# Create Interactive Quality Assessment Interface

# Global state
assessment_grid = None
quality_results = {}
quality_metrics = {}
current_model_id = None
current_grid_id = None
loaded_grid_data = None
signal_arrays = {}

# ============================================
# Helper Functions for Demo Data
# ============================================

def generate_sample_grid_with_quality_issues():
    """Generate sample voxel grid with quality issues."""
    np.random.seed(42)
    
    # Create a 3D grid
    x = np.linspace(-50, 50, 50)
    y = np.linspace(-50, 50, 50)
    z = np.linspace(0, 100, 50)
    X, Y, Z = np.meshgrid(x, y, z, indexing='ij')
    
    # Create signal with some gaps
    signal = 100 + 50 * np.sin(2 * np.pi * X / 20) * np.cos(2 * np.pi * Y / 20)
    signal += 20 * np.sin(2 * np.pi * Z / 10)
    
    # Add noise
    noise = np.random.normal(0, 5, signal.shape)
    signal += noise
    
    # Create gaps (missing regions)
    gap_mask = np.random.random(signal.shape) < 0.15  # 15% gaps
    signal[gap_mask] = np.nan
    
    # Add some low-quality regions
    low_quality_mask = np.random.random(signal.shape) < 0.1  # 10% low quality
    signal[low_quality_mask] += np.random.normal(0, 20, np.sum(low_quality_mask))
    
    return {
        'signal': signal,
        'grid_shape': signal.shape,
        'coords': (X, Y, Z)
    }

# ============================================
# Top Panel: Data Source and Grid Selection
# ============================================

# Data source mode
data_source_label = widgets.HTML("<b>Data Source:</b>")
data_source_mode = RadioButtons(
    options=[('MongoDB', 'mongodb'), ('Sample Data', 'sample')],
    value='mongodb',
    description='Mode:',
    style={'description_width': 'initial'}
)

# Model selection (for MongoDB)
model_label = widgets.HTML("<b>Model:</b>")
model_options = [("━━━ Select Model ━━━", None)]
if stl_client and mongo_client:
    try:
        models = stl_client.list_models(limit=100)
        model_options.extend([
            (f"{m.get('filename', m.get('original_stem', m.get('model_name', 'Unknown')))} ({m.get('model_id', '')[:8]}...)", m.get('model_id'))
            for m in models
        ])
    except Exception as e:
        print(f"⚠️ Error loading models: {e}")

model_dropdown = Dropdown(
    options=model_options,
    value=None,
    description='Model:',
    style={'description_width': 'initial'},
    layout=Layout(width='400px')
)

# Grid type filter
grid_type_label = widgets.HTML("<b>Grid Type:</b>")
grid_type_filter = Dropdown(
    options=[
        ('All Grids', 'all'),
        ('Fused', 'fused'),
        ('Corrected', 'corrected'),
        ('Processed', 'processed'),
        ('Signal-Mapped', 'signal_mapped'),
        ('Raw', 'raw')
    ],
    value='fused',  # Default to fused grids
    description='Type:',
    style={'description_width': 'initial'}
)

# Grid selection (for MongoDB)
grid_label = widgets.HTML("<b>Grid:</b>")
grid_dropdown = Dropdown(
    options=[("━━━ Select Grid ━━━", None)],
    value=None,
    description='Grid:',
    style={'description_width': 'initial'},
    layout=Layout(width='500px')
)

load_grid_button = Button(
    description='Load Grid',
    button_style='info',
    icon='folder-open',
    layout=Layout(width='120px')
)

# Assessment type
assessment_type = RadioButtons(
    options=[
        ('Data Quality', 'data'),
        ('Signal Quality', 'signal'),
        ('Alignment', 'alignment'),
        ('Completeness', 'completeness'),
        ('All', 'all')
    ],
    value='all',  # Default to 'all' for comprehensive assessment
    description='Assessment:',
    style={'description_width': 'initial'}
)

execute_button = Button(
    description='Execute Assessment',
    button_style='success',
    icon='check',
    layout=Layout(width='180px')
)

export_button = Button(
    description='Export Report',
    button_style='',
    icon='download',
    layout=Layout(width='150px')
)

top_panel = VBox([
    HBox([data_source_label, data_source_mode]),
    HBox([model_label, model_dropdown, grid_type_label, grid_type_filter]),
    HBox([grid_label, grid_dropdown, load_grid_button]),
    HBox([assessment_type, execute_button, export_button])
], layout=Layout(padding='10px', border='1px solid #ccc'))

# ============================================
# Left Panel: Assessment Configuration
# ============================================

# Data Quality Section
data_quality_label = widgets.HTML("<b>Data Quality:</b>")
metrics_completeness = Checkbox(value=True, description='Completeness', style={'description_width': 'initial'})
metrics_coverage = Checkbox(value=True, description='Coverage', style={'description_width': 'initial'})
metrics_consistency = Checkbox(value=True, description='Consistency', style={'description_width': 'initial'})
metrics_accuracy = Checkbox(value=False, description='Accuracy', style={'description_width': 'initial'})
metrics_reliability = Checkbox(value=False, description='Reliability', style={'description_width': 'initial'})

completeness_threshold = FloatSlider(value=0.8, min=0.0, max=1.0, step=0.05, description='Completeness Threshold:', style={'description_width': 'initial'})
coverage_type = RadioButtons(
    options=[('Spatial', 'spatial'), ('Temporal', 'temporal'), ('Both', 'both')],
    value='both',
    description='Coverage:',
    style={'description_width': 'initial'}
)

data_quality_section = VBox([
    data_quality_label,
    metrics_completeness,
    metrics_coverage,
    metrics_consistency,
    metrics_accuracy,
    metrics_reliability,
    completeness_threshold,
    coverage_type
], layout=Layout(padding='5px', border='1px solid #ddd'))

# Signal Quality Section
signal_quality_label = widgets.HTML("<b>Signal Quality:</b>")
signal_selector = Dropdown(
    options=[('Temperature', 'temperature'), ('Power', 'power'), ('Density', 'density')],
    value='temperature',
    description='Signal:',
    style={'description_width': 'initial'}
)
snr_threshold = FloatSlider(value=10.0, min=0.0, max=100.0, step=1.0, description='SNR Threshold (dB):', style={'description_width': 'initial'})
uncertainty_threshold = FloatSlider(value=0.1, min=0.0, max=1.0, step=0.05, description='Uncertainty Threshold:', style={'description_width': 'initial'})
confidence_threshold = FloatSlider(value=0.7, min=0.0, max=1.0, step=0.05, description='Confidence Threshold:', style={'description_width': 'initial'})

signal_quality_section = VBox([
    signal_quality_label,
    signal_selector,
    snr_threshold,
    uncertainty_threshold,
    confidence_threshold
], layout=Layout(padding='5px', border='1px solid #ddd'))

# Alignment Section
alignment_label = widgets.HTML("<b>Alignment:</b>")
max_error = FloatSlider(value=0.1, min=0.01, max=10.0, step=0.01, description='Max Error (mm):', style={'description_width': 'initial'})
temporal_tolerance = FloatSlider(value=1.0, min=0.1, max=10.0, step=0.1, description='Temporal Tolerance (s):', style={'description_width': 'initial'})
spatial_tolerance = FloatSlider(value=0.1, min=0.01, max=10.0, step=0.01, description='Spatial Tolerance (mm):', style={'description_width': 'initial'})

alignment_section = VBox([
    alignment_label,
    max_error,
    temporal_tolerance,
    spatial_tolerance
], layout=Layout(padding='5px', border='1px solid #ddd'))

# Completeness Section
completeness_label = widgets.HTML("<b>Completeness:</b>")
gap_detection = Checkbox(value=True, description='Gap Detection', style={'description_width': 'initial'})
gap_size_threshold = IntSlider(value=10, min=1, max=1000, step=1, description='Gap Size Threshold:', style={'description_width': 'initial'})
fill_gaps = Checkbox(value=False, description='Fill Gaps', style={'description_width': 'initial'})
gap_filling_strategy = Dropdown(
    options=[('Interpolation', 'interpolation'), ('Nearest', 'nearest'), ('Zero', 'zero')],
    value='interpolation',
    description='Filling Strategy:',
    style={'description_width': 'initial'}
)

completeness_section = VBox([
    completeness_label,
    gap_detection,
    gap_size_threshold,
    fill_gaps,
    gap_filling_strategy
], layout=Layout(padding='5px', border='1px solid #ddd'))

# Show/hide sections based on assessment type
def update_assessment_sections(change):
    """Show/hide assessment sections based on type."""
    assess_type = change['new']
    data_quality_section.layout.display = 'none'
    signal_quality_section.layout.display = 'none'
    alignment_section.layout.display = 'none'
    completeness_section.layout.display = 'none'
    
    if assess_type == 'data' or assess_type == 'all':
        data_quality_section.layout.display = 'flex'
    if assess_type == 'signal' or assess_type == 'all':
        signal_quality_section.layout.display = 'flex'
    if assess_type == 'alignment' or assess_type == 'all':
        alignment_section.layout.display = 'flex'
    if assess_type == 'completeness' or assess_type == 'all':
        completeness_section.layout.display = 'flex'

assessment_type.observe(update_assessment_sections, names='value')
update_assessment_sections({'new': assessment_type.value})

left_panel = VBox([
    data_quality_section,
    signal_quality_section,
    alignment_section,
    completeness_section
], layout=Layout(width='300px', padding='10px', border='1px solid #ccc'))

# ============================================
# Center Panel: Visualization
# ============================================

viz_mode = RadioButtons(
    options=[('Quality Map', 'quality_map'), ('Metrics', 'metrics'), ('Gaps', 'gaps'), ('Trends', 'trends')],
    value='quality_map',
    description='View:',
    style={'description_width': 'initial'}
)

# Quality map controls
colormap_selector = Dropdown(
    options=[('Viridis', 'viridis'), ('RdYlGn', 'RdYlGn'), ('Coolwarm', 'coolwarm'), ('Plasma', 'plasma')],
    value='RdYlGn',
    description='Colormap:',
    style={'description_width': 'initial'}
)
quality_threshold_overlay = Checkbox(value=True, description='Show Threshold', style={'description_width': 'initial'})

viz_output = Output(layout=Layout(height='500px', overflow='auto'))

center_panel = VBox([
    widgets.HTML("<h3>Quality Visualization</h3>"),
    viz_mode,
    colormap_selector,
    quality_threshold_overlay,
    viz_output
], layout=Layout(flex='1 1 auto', padding='10px', border='1px solid #ccc'))

# ============================================
# Right Panel: Results and Metrics
# ============================================

# Overall Quality Score
overall_score_label = widgets.HTML("<b>Overall Quality Score:</b>")
overall_score_display = widgets.HTML("<h2 style='color: green;'>0.85</h2>")
overall_score_section = VBox([
    overall_score_label,
    overall_score_display
], layout=Layout(padding='5px', border='2px solid #4CAF50'))

# Quality Metrics Table
metrics_table_label = widgets.HTML("<b>Quality Metrics:</b>")
metrics_table_display = widgets.HTML("No metrics available")
metrics_table_section = VBox([
    metrics_table_label,
    metrics_table_display
], layout=Layout(padding='5px'))

# Signal Quality Metrics
signal_quality_metrics_label = widgets.HTML("<b>Signal Quality:</b>")
signal_quality_metrics_display = widgets.HTML("No signal metrics available")
signal_quality_metrics_section = VBox([
    signal_quality_metrics_label,
    signal_quality_metrics_display
], layout=Layout(padding='5px'))

# Alignment Metrics
alignment_metrics_label = widgets.HTML("<b>Alignment Metrics:</b>")
alignment_metrics_display = widgets.HTML("No alignment metrics available")
alignment_metrics_section = VBox([
    alignment_metrics_label,
    alignment_metrics_display
], layout=Layout(padding='5px'))

# Completeness Metrics
completeness_metrics_label = widgets.HTML("<b>Completeness:</b>")
completeness_metrics_display = widgets.HTML("No completeness metrics available")
completeness_metrics_section = VBox([
    completeness_metrics_label,
    completeness_metrics_display
], layout=Layout(padding='5px'))

# Quality Status
quality_status_label = widgets.HTML("<b>Quality Status:</b>")
quality_status_display = widgets.HTML("Not assessed")
quality_status_section = VBox([
    quality_status_label,
    quality_status_display
], layout=Layout(padding='5px'))

# Export Options
export_label = widgets.HTML("<b>Export:</b>")
export_report_button = Button(description='Export Report', button_style='', layout=Layout(width='150px'))
export_quality_map_button = Button(description='Export Quality Map', button_style='', layout=Layout(width='150px'))
export_metrics_button = Button(description='Export Metrics', button_style='', layout=Layout(width='150px'))
save_assessment_button = Button(description='Save Assessment', button_style='success', layout=Layout(width='150px'))
save_config_button = Button(description='Save Config', button_style='', layout=Layout(width='150px'))

export_section = VBox([
    export_label,
    export_report_button,
    export_quality_map_button,
    export_metrics_button,
    save_assessment_button,
    save_config_button
], layout=Layout(padding='5px'))

right_panel = VBox([
    overall_score_section,
    metrics_table_section,
    signal_quality_metrics_section,
    alignment_metrics_section,
    completeness_metrics_section,
    quality_status_section,
    export_section
], layout=Layout(width='250px', padding='10px', border='1px solid #ccc'))

# ============================================
# Bottom Panel: Status and Progress
# ============================================

status_display = widgets.HTML("<b>Status:</b> Ready to assess quality")
progress_bar = widgets.IntProgress(
    value=0,
    min=0,
    max=100,
    description='Progress:',
    bar_style='info',
    layout=Layout(width='100%')
)
warning_display = widgets.HTML("")

bottom_panel = VBox([
    status_display,
    progress_bar,
    warning_display
], layout=Layout(padding='10px', border='1px solid #ccc'))

# ============================================
# Helper Functions for MongoDB
# ============================================

def update_grid_dropdown(change=None):
    """Update grid dropdown when model or grid type changes."""
    global current_model_id
    
    model_id = model_dropdown.value
    grid_type = grid_type_filter.value
    
    if not model_id:
        grid_dropdown.options = [("━━━ Select Grid ━━━", None)]
        return
    
    current_model_id = model_id
    
    if not voxel_storage:
        grid_dropdown.options = [("━━━ MongoDB not available ━━━", None)]
        return
    
    try:
        # Get all grids for this model
        grids = voxel_storage.list_grids(model_id=model_id, limit=100)
        
        grid_options = [("━━━ Select Grid ━━━", None)]
        for grid in grids:
            metadata = grid.get('metadata', {})
            config_meta = metadata.get('configuration_metadata', {})
            if not config_meta:
                config_meta = metadata
            
            # Determine grid type
            is_fused = config_meta.get('fusion_applied', False)
            is_corrected = config_meta.get('correction_applied', False)
            is_processed = config_meta.get('processing_applied', False)
            has_signals = len(grid.get('available_signals', [])) > 0
            
            grid_type_match = False
            if grid_type == 'all':
                grid_type_match = True
            elif grid_type == 'fused' and is_fused:
                grid_type_match = True
            elif grid_type == 'corrected' and is_corrected:
                grid_type_match = True
            elif grid_type == 'processed' and is_processed:
                grid_type_match = True
            elif grid_type == 'signal_mapped' and has_signals and not is_corrected and not is_processed and not is_fused:
                grid_type_match = True
            elif grid_type == 'raw' and not has_signals:
                grid_type_match = True
            
            if grid_type_match:
                grid_id = grid.get('grid_id', str(grid.get('_id', '')))
                grid_name = grid.get('grid_name', 'Unknown')
                n_signals = len(grid.get('available_signals', []))
                
                # Build status label
                status_parts = []
                if is_fused:
                    status_parts.append('fused')
                if is_corrected:
                    status_parts.append('corrected')
                if is_processed:
                    status_parts.append('processed')
                if has_signals and not status_parts:
                    status_parts.append('mapped')
                if not status_parts:
                    status_parts.append('raw')
                
                status_str = ', '.join(status_parts)
                label = f"{grid_name} ({n_signals} signal(s), {status_str}) ({grid_id[:8]}...)"
                grid_options.append((label, grid_id))
        
        if len(grid_options) == 1:
            grid_options.append(("No grids found matching filter", None))
        
        grid_dropdown.options = grid_options
    except Exception as e:
        grid_dropdown.options = [("━━━ Error loading grids ━━━", None)]
        print(f"⚠️ Error loading grids: {e}")

def load_grid_from_mongodb(button):
    """Load selected grid from MongoDB."""
    global assessment_grid, current_model_id, current_grid_id, loaded_grid_data, signal_arrays
    
    if not voxel_storage or not grid_dropdown.value:
        warning_display.value = "<span style='color: red;'>⚠️ Please select a grid to load</span>"
        return
    
    grid_id = grid_dropdown.value
    current_grid_id = grid_id
    
    status_display.value = "<b>Status:</b> Loading grid from MongoDB..."
    progress_bar.value = 0
    warning_display.value = ""
    
    try:
        # Load grid from MongoDB
        grid_data = voxel_storage.load_voxel_grid(grid_id=grid_id)
        
        if not grid_data:
            warning_display.value = "<span style='color: red;'>⚠️ Failed to load grid</span>"
            return
        
        # Extract data from dictionary
        signal_arrays = grid_data.get('signal_arrays', {})
        metadata = grid_data.get('metadata', {})
        grid_name = grid_data.get('grid_name', 'Unknown')
        
        if not signal_arrays or len(signal_arrays) == 0:
            warning_display.value = "<span style='color: orange;'>⚠️ Grid has no signals - will assess grid structure only</span>"
            # Still allow assessment of grid structure
        
        progress_bar.value = 50
        
        # Reconstruct VoxelGrid from metadata
        from am_qadf.voxelization.voxel_grid import VoxelGrid
        
        # Get grid properties from metadata
        bbox_min = metadata.get('bbox_min', [-50, -50, 0])
        bbox_max = metadata.get('bbox_max', [50, 50, 100])
        resolution = metadata.get('resolution', 1.0)
        
        # Handle resolution - can be a list or single float
        if isinstance(resolution, (list, tuple, np.ndarray)):
            resolution = float(np.mean(resolution))
        else:
            resolution = float(resolution)
        
        # Ensure bbox_min and bbox_max are tuples/lists
        if not isinstance(bbox_min, (list, tuple, np.ndarray)):
            bbox_min = [-50, -50, 0]
        if not isinstance(bbox_max, (list, tuple, np.ndarray)):
            bbox_max = [50, 50, 100]
        
        # Convert to tuples
        bbox_min = tuple(bbox_min[:3])
        bbox_max = tuple(bbox_max[:3])
        
        # Create VoxelGrid object
        grid = VoxelGrid(bbox_min=bbox_min, bbox_max=bbox_max, resolution=resolution)
        
        # Add signals to grid
        if not hasattr(grid, '_signal_arrays'):
            grid._signal_arrays = {}
        for signal_name, signal_array in signal_arrays.items():
            grid._signal_arrays[signal_name] = signal_array
        
        if not hasattr(grid, 'available_signals'):
            grid.available_signals = set()
        grid.available_signals.update(signal_arrays.keys())
        
        # Add get_signal_array method
        def get_signal_array(signal_name, default=0.0):
            if hasattr(grid, '_signal_arrays') and signal_name in grid._signal_arrays:
                return grid._signal_arrays[signal_name]
            return None
        grid.get_signal_array = get_signal_array
        
        # Store loaded data
        loaded_grid_data = {
            'grid': grid,
            'grid_data': grid_data,
            'metadata': metadata,
            'signal_arrays': signal_arrays
        }
        
        # Update signal selector with available signals
        if signal_arrays:
            signal_options = [("━━━ All Signals ━━━", 'all')]
            signal_options.extend([(name, name) for name in sorted(signal_arrays.keys())])
            signal_selector.options = signal_options
            if len(signal_options) > 1:
                signal_selector.value = signal_options[1][1]  # Select first signal
        
        progress_bar.value = 100
        status_display.value = f"<b>Status:</b> <span style='color: green;'>✅ Loaded grid: {grid_name} ({len(signal_arrays)} signal(s))</span>"
        
    except Exception as e:
        warning_display.value = f"<span style='color: red;'>❌ Error loading grid: {str(e)}</span>"
        status_display.value = f"<b>Status:</b> <span style='color: red;'>Error loading grid</span>"
        progress_bar.value = 0
        import traceback
        traceback.print_exc()

# Function to update UI based on data source mode
def update_data_source_mode(change):
    """Show/hide MongoDB widgets based on data source mode."""
    if change['new'] == 'mongodb':
        model_dropdown.layout.display = 'flex'
        grid_type_filter.layout.display = 'flex'
        grid_dropdown.layout.display = 'flex'
        load_grid_button.layout.display = 'flex'
    else:
        model_dropdown.layout.display = 'none'
        grid_type_filter.layout.display = 'none'
        grid_dropdown.layout.display = 'none'
        load_grid_button.layout.display = 'none'

# Connect events
data_source_mode.observe(update_data_source_mode, names='value')
update_data_source_mode({'new': data_source_mode.value})
model_dropdown.observe(update_grid_dropdown, names='value')
grid_type_filter.observe(update_grid_dropdown, names='value')
load_grid_button.on_click(load_grid_from_mongodb)

# ============================================
# Assessment Functions
# ============================================

def execute_assessment(button):
    """Execute quality assessment based on current settings."""
    global assessment_grid, quality_results, quality_metrics, loaded_grid_data, signal_arrays
    
    status_display.value = "<b>Status:</b> Assessing quality..."
    progress_bar.value = 0
    warning_display.value = ""
    
    try:
        # Load data based on mode
        if data_source_mode.value == 'mongodb':
            if not loaded_grid_data:
                warning_display.value = "<span style='color: red;'>⚠️ Please load a grid from MongoDB first</span>"
                return
            
            # Use loaded grid data
            grid = loaded_grid_data['grid']
            metadata = loaded_grid_data['metadata']
            grid_data = loaded_grid_data['grid_data']
            
            # Get signal for assessment
            selected_signal = signal_selector.value if signal_selector.value and signal_selector.value != 'all' else None
            if signal_arrays and selected_signal and selected_signal in signal_arrays:
                signal_array = signal_arrays[selected_signal]
            elif signal_arrays and len(signal_arrays) > 0:
                # Use first available signal
                signal_array = list(signal_arrays.values())[0]
            else:
                # No signals - create empty signal for structure assessment
                signal_array = np.zeros(grid.dims, dtype=np.float32)
            
            assessment_grid = {
                'signal': signal_array,
                'grid_shape': signal_array.shape,
                'grid': grid,
                'metadata': metadata,
                'grid_data': grid_data
            }
            progress_bar.value = 20
        else:
            # Use sample data
            assessment_grid = generate_sample_grid_with_quality_issues()
            progress_bar.value = 20
        
        assess_type = assessment_type.value
        
        # Extract existing metadata for enhanced assessment
        existing_metadata = {}
        if data_source_mode.value == 'mongodb' and 'metadata' in assessment_grid:
            metadata = assessment_grid['metadata']
            config_meta = metadata.get('configuration_metadata', {})
            if not config_meta:
                config_meta = metadata
            
            # Extract fusion metrics (if fused grid)
            if config_meta.get('fusion_applied', False):
                existing_metadata['fusion'] = config_meta.get('fusion_metrics', {})
            
            # Extract correction metrics (if corrected grid)
            if config_meta.get('correction_applied', False):
                existing_metadata['correction'] = config_meta.get('correction_metrics', {})
            
            # Extract processing metrics (if processed grid)
            if config_meta.get('processing_applied', False):
                existing_metadata['processing'] = config_meta.get('processing_metrics', {})
        
        # Data quality assessment
        if assess_type == 'data' or assess_type == 'all':
            # Calculate completeness
            signal = assessment_grid['signal']
            total_voxels = signal.size
            filled_voxels = np.sum(~np.isnan(signal)) if hasattr(signal, 'dtype') and np.issubdtype(signal.dtype, np.floating) else np.sum(signal != 0)
            completeness = filled_voxels / total_voxels if total_voxels > 0 else 0
            
            # Calculate coverage (spatial)
            nan_count = np.sum(np.isnan(signal)) if hasattr(signal, 'dtype') and np.issubdtype(signal.dtype, np.floating) else 0
            coverage_spatial = 1.0 - (nan_count / total_voxels)
            
            # Use existing metrics if available, otherwise calculate
            if 'fusion' in existing_metadata:
                fusion_metrics = existing_metadata['fusion']
                consistency = fusion_metrics.get('consistency_score', 0.85)
                coverage_temporal = fusion_metrics.get('coverage', 0.90) / 100.0 if fusion_metrics.get('coverage', 0) > 1 else fusion_metrics.get('coverage', 0.90)
            elif 'correction' in existing_metadata:
                correction_metrics = existing_metadata['correction']
                consistency = correction_metrics.get('score', 0.85)
                coverage_temporal = 0.90  # Default
            else:
                consistency = 0.85  # Default
                coverage_temporal = 0.90  # Default
            
            # Calculate accuracy and reliability from signal statistics
            if signal.size > 0:
                signal_clean = signal[~np.isnan(signal)] if hasattr(signal, 'dtype') and np.issubdtype(signal.dtype, np.floating) else signal[signal != 0]
                if len(signal_clean) > 0:
                    # Accuracy based on signal consistency
                    accuracy = 1.0 - (np.std(signal_clean) / (np.abs(np.mean(signal_clean)) + 1e-10))
                    accuracy = max(0.0, min(1.0, accuracy))
                    # Reliability based on completeness and consistency
                    reliability = (completeness + consistency) / 2.0
                else:
                    accuracy = 0.0
                    reliability = 0.0
            else:
                accuracy = 0.0
                reliability = 0.0
            
            quality_metrics['data'] = {
                'completeness': completeness,
                'coverage_spatial': coverage_spatial,
                'coverage_temporal': coverage_temporal,
                'consistency': consistency,
                'accuracy': accuracy,
                'reliability': reliability
            }
            progress_bar.value = 40
        
        # Signal quality assessment
        if assess_type == 'signal' or assess_type == 'all':
            signal = assessment_grid['signal']
            signal_clean = signal[~np.isnan(signal)] if hasattr(signal, 'dtype') and np.issubdtype(signal.dtype, np.floating) else signal[signal != 0]
            
            if len(signal_clean) > 0:
                # Calculate SNR
                mean_signal = np.mean(signal_clean)
                std_signal = np.std(signal_clean)
                snr = mean_signal / std_signal if std_signal > 0 else 0
                snr_db = 20 * np.log10(snr) if snr > 0 else 0
                
                # Calculate uncertainty (simplified)
                uncertainty = std_signal / mean_signal if mean_signal > 0 else 1.0
                
                # Calculate confidence (inverse of uncertainty)
                confidence = 1.0 / (1.0 + uncertainty)
                
                # Use processing metrics if available
                if 'processing' in existing_metadata:
                    proc_metrics = existing_metadata['processing']
                    if 'processed_snr' in proc_metrics:
                        snr_db = proc_metrics.get('processed_snr', snr_db)
                    if 'quality_score' in proc_metrics:
                        confidence = proc_metrics.get('quality_score', confidence)
            else:
                snr_db = 0.0
                uncertainty = 1.0
                confidence = 0.0
                mean_signal = 0.0
                std_signal = 0.0
            
            quality_metrics['signal'] = {
                'snr': snr_db,
                'uncertainty': uncertainty,
                'confidence': confidence,
                'mean': float(mean_signal) if 'mean_signal' in locals() else 0.0,
                'std': float(std_signal) if 'std_signal' in locals() else 0.0
            }
            progress_bar.value = 60
        
        # Alignment assessment
        if assess_type == 'alignment' or assess_type == 'all':
            # Try to get alignment metrics from metadata
            if data_source_mode.value == 'mongodb' and 'metadata' in assessment_grid:
                metadata = assessment_grid['metadata']
                # Check if grid was created from alignment
                alignment_id = metadata.get('alignment_id') or metadata.get('configuration_metadata', {}).get('alignment_id')
                
                if alignment_id:
                    # Try to load alignment data
                    try:
                        from am_qadf.synchronization import AlignmentStorage
                        alignment_storage = AlignmentStorage(mongo_client=mongo_client)
                        alignment_data = alignment_storage.load_alignment(alignment_id=alignment_id, load_data=False)
                        
                        if alignment_data:
                            align_metrics = alignment_data.get('alignment_metrics', {})
                            quality_metrics['alignment'] = {
                                'coordinate_accuracy': align_metrics.get('spatial_accuracy', {}).get('mean_error', 0.05),
                                'temporal_accuracy': align_metrics.get('temporal_accuracy', {}).get('mean_error', 0.5),
                                'spatial_accuracy': align_metrics.get('spatial_accuracy', {}).get('rms_error', 0.08),
                                'alignment_score': align_metrics.get('score', 0.85),
                                'validation_status': align_metrics.get('validation_status', 'unknown')
                            }
                        else:
                            # Use default values
                            quality_metrics['alignment'] = {
                                'coordinate_accuracy': 0.05,
                                'temporal_accuracy': 0.5,
                                'spatial_accuracy': 0.08
                            }
                    except Exception as e:
                        # Fallback to default values
                        quality_metrics['alignment'] = {
                            'coordinate_accuracy': 0.05,
                            'temporal_accuracy': 0.5,
                            'spatial_accuracy': 0.08
                        }
                else:
                    # No alignment data - use default values
                    quality_metrics['alignment'] = {
                        'coordinate_accuracy': 0.05,
                        'temporal_accuracy': 0.5,
                        'spatial_accuracy': 0.08
                    }
            else:
                # Sample mode - use default values
                quality_metrics['alignment'] = {
                    'coordinate_accuracy': 0.05,
                    'temporal_accuracy': 0.5,
                    'spatial_accuracy': 0.08
                }
            progress_bar.value = 70
        
        # Completeness assessment
        if assess_type == 'completeness' or assess_type == 'all':
            signal = assessment_grid['signal']
            gap_mask = np.isnan(signal) if hasattr(signal, 'dtype') and np.issubdtype(signal.dtype, np.floating) else (signal == 0)
            gap_count = np.sum(gap_mask)
            total_voxels = signal.size
            coverage = 1.0 - (gap_count / total_voxels) if total_voxels > 0 else 0.0
            
            # Identify gap regions (simplified)
            gap_regions = []
            if gap_detection.value and gap_count > 0:
                # Find connected gap regions (simplified - just count gaps)
                gap_regions = [{'size': int(gap_count), 'bbox': 'N/A', 'percentage': float(gap_count / total_voxels * 100)}]
            
            quality_metrics['completeness'] = {
                'coverage': coverage,
                'gap_count': int(gap_count),
                'total_voxels': int(total_voxels),
                'gap_regions': gap_regions,
                'gap_percentage': float(gap_count / total_voxels * 100) if total_voxels > 0 else 0.0
            }
            progress_bar.value = 80
        
        # Calculate overall quality score
        if quality_metrics:
            scores = []
            if 'data' in quality_metrics:
                scores.append(quality_metrics['data'].get('completeness', 0))
            if 'signal' in quality_metrics:
                scores.append(quality_metrics['signal'].get('confidence', 0))
            if 'alignment' in quality_metrics:
                # Convert accuracy to score (lower is better)
                acc = quality_metrics['alignment'].get('spatial_accuracy', 1.0)
                scores.append(max(0, 1.0 - acc / max_error.value))
            if 'completeness' in quality_metrics:
                scores.append(quality_metrics['completeness'].get('coverage', 0))
            
            overall_score = np.mean(scores) if scores else 0.0
        else:
            overall_score = 0.0
        
        quality_results = {
            'overall_score': overall_score,
            'metrics': quality_metrics
        }
        
        progress_bar.value = 90
        
        # Update displays
        update_results_display()
        update_visualization()
        
        progress_bar.value = 100
        status_display.value = "<b>Status:</b> <span style='color: green;'>✅ Assessment completed successfully</span>"
        
        # Add warnings if needed
        warnings = []
        if overall_score < 0.7:
            warnings.append("⚠️ Overall quality score is below 0.7")
        if 'data' in quality_metrics and quality_metrics['data'].get('completeness', 1.0) < completeness_threshold.value:
            warnings.append(f"⚠️ Completeness ({quality_metrics['data']['completeness']:.2f}) below threshold ({completeness_threshold.value:.2f})")
        if 'signal' in quality_metrics and quality_metrics['signal'].get('snr', 100) < snr_threshold.value:
            warnings.append(f"⚠️ SNR ({quality_metrics['signal']['snr']:.1f} dB) below threshold ({snr_threshold.value:.1f} dB)")
        
        if warnings:
            warning_display.value = "<br>".join([f"<span style='color: orange;'>{w}</span>" for w in warnings])
        
    except Exception as e:
        warning_display.value = f"<span style='color: red;'>❌ Error: {str(e)}</span>"
        status_display.value = f"<b>Status:</b> <span style='color: red;'>Error during assessment</span>"
        progress_bar.value = 0

def update_results_display():
    """Update results and metrics displays."""
    global quality_results, quality_metrics
    
    if not quality_results:
        return
    
    # Overall score
    overall_score = quality_results.get('overall_score', 0.0)
    color = 'green' if overall_score >= 0.7 else 'orange' if overall_score >= 0.5 else 'red'
    overall_score_display.value = f"<h2 style='color: {color};'>{overall_score:.2f}</h2>"
    
    # Quality metrics table
    if 'data' in quality_metrics:
        data = quality_metrics['data']
        metrics_html = f"""
        <table border='1' style='border-collapse: collapse; width: 100%;'>
        <tr><th>Metric</th><th>Value</th></tr>
        <tr><td>Completeness</td><td>{data.get('completeness', 0):.2f}</td></tr>
        <tr><td>Coverage (Spatial)</td><td>{data.get('coverage_spatial', 0):.2f}</td></tr>
        <tr><td>Coverage (Temporal)</td><td>{data.get('coverage_temporal', 0):.2f}</td></tr>
        <tr><td>Consistency</td><td>{data.get('consistency', 0):.2f}</td></tr>
        <tr><td>Accuracy</td><td>{data.get('accuracy', 0):.2f}</td></tr>
        <tr><td>Reliability</td><td>{data.get('reliability', 0):.2f}</td></tr>
        </table>
        """
        metrics_table_display.value = metrics_html
    
    # Signal quality metrics
    if 'signal' in quality_metrics:
        signal = quality_metrics['signal']
        signal_html = f"""
        <p><b>Mean SNR:</b> {signal.get('snr', 0):.1f} dB</p>
        <p><b>Mean Uncertainty:</b> {signal.get('uncertainty', 0):.3f}</p>
        <p><b>Mean Confidence:</b> {signal.get('confidence', 0):.2f}</p>
        """
        signal_quality_metrics_display.value = signal_html
    
    # Alignment metrics
    if 'alignment' in quality_metrics:
        align = quality_metrics['alignment']
        align_html = f"""
        <p><b>Coordinate Accuracy:</b> {align.get('coordinate_accuracy', 0):.3f} mm</p>
        <p><b>Temporal Accuracy:</b> {align.get('temporal_accuracy', 0):.2f} s</p>
        <p><b>Spatial Accuracy:</b> {align.get('spatial_accuracy', 0):.3f} mm</p>
        """
        alignment_metrics_display.value = align_html
    
    # Completeness metrics
    if 'completeness' in quality_metrics:
        comp = quality_metrics['completeness']
        comp_html = f"""
        <p><b>Coverage:</b> {comp.get('coverage', 0):.1f}%</p>
        <p><b>Gap Count:</b> {comp.get('gap_count', 0):,}</p>
        <p><b>Gap Regions:</b> {len(comp.get('gap_regions', []))}</p>
        """
        completeness_metrics_display.value = comp_html
    
    # Quality status
    if overall_score >= 0.8:
        status_html = "<p style='color: green;'>✅ <b>Pass</b> - High Quality</p>"
    elif overall_score >= 0.6:
        status_html = "<p style='color: orange;'>⚠️ <b>Warning</b> - Moderate Quality</p>"
    else:
        status_html = "<p style='color: red;'>❌ <b>Fail</b> - Low Quality</p>"
    quality_status_display.value = status_html

def update_visualization():
    """Update visualization display."""
    global assessment_grid, quality_results
    
    with viz_output:
        clear_output(wait=True)
        
        if assessment_grid is None:
            display(HTML("<p>Execute assessment to see visualization</p>"))
            return
        
        mode = viz_mode.value
        signal = assessment_grid['signal']
        slice_idx = signal.shape[2] // 2
        
        if mode == 'quality_map':
            fig, ax = plt.subplots(figsize=(10, 8))
            
            # Create quality map (1.0 = high quality, 0.0 = low quality)
            quality_map = np.ones_like(signal)
            # Lower quality for NaN regions
            quality_map[np.isnan(signal)] = 0.0
            # Lower quality for regions with high variance (simplified)
            if signal.size > 0:
                signal_clean = signal[~np.isnan(signal)]
                if len(signal_clean) > 0:
                    mean_sig = np.nanmean(signal)
                    std_sig = np.nanstd(signal)
                    # Regions far from mean have lower quality
                    quality_map[~np.isnan(signal)] = 1.0 - np.abs(signal[~np.isnan(signal)] - mean_sig) / (3 * std_sig + 1e-10)
                    quality_map = np.clip(quality_map, 0.0, 1.0)
            
            im = ax.imshow(quality_map[:, :, slice_idx], cmap=colormap_selector.value, origin='lower', vmin=0, vmax=1)
            ax.set_title('Quality Map')
            ax.set_xlabel('X')
            ax.set_ylabel('Y')
            plt.colorbar(im, ax=ax, label='Quality Score')
            
            if quality_threshold_overlay.value and quality_results:
                threshold = quality_results.get('overall_score', 0.7)
                ax.contour(quality_map[:, :, slice_idx], levels=[threshold], colors='red', linewidths=2)
            
            plt.tight_layout()
            plt.show()
        
        elif mode == 'metrics':
            fig, axes = plt.subplots(2, 2, figsize=(12, 10))
            
            # Quality score gauge
            ax1 = axes[0, 0]
            if quality_results:
                score = quality_results.get('overall_score', 0.0)
                ax1.barh([0], [score], color='green' if score >= 0.7 else 'orange' if score >= 0.5 else 'red')
                ax1.set_xlim(0, 1)
                ax1.set_xlabel('Quality Score')
                ax1.set_title('Overall Quality Score')
                ax1.set_yticks([])
            
            # Metric comparison
            ax2 = axes[0, 1]
            if 'data' in quality_metrics:
                data = quality_metrics['data']
                metrics = ['Completeness', 'Coverage', 'Consistency', 'Accuracy']
                values = [
                    data.get('completeness', 0),
                    data.get('coverage_spatial', 0),
                    data.get('consistency', 0),
                    data.get('accuracy', 0)
                ]
                ax2.bar(metrics, values, color=['blue', 'green', 'orange', 'red'])
                ax2.set_ylim(0, 1)
                ax2.set_ylabel('Score')
                ax2.set_title('Quality Metrics Comparison')
                ax2.tick_params(axis='x', rotation=45)
            
            # Quality distribution
            ax3 = axes[1, 0]
            if assessment_grid:
                signal_clean = signal[~np.isnan(signal)]
                ax3.hist(signal_clean.flatten(), bins=50, alpha=0.7, edgecolor='black')
                ax3.set_xlabel('Signal Value')
                ax3.set_ylabel('Frequency')
                ax3.set_title('Signal Distribution')
            
            # SNR plot
            ax4 = axes[1, 1]
            if 'signal' in quality_metrics:
                signal_data = quality_metrics['signal']
                snr = signal_data.get('snr', 0)
                ax4.bar(['SNR'], [snr], color='blue')
                ax4.axhline(y=snr_threshold.value, color='r', linestyle='--', label=f'Threshold ({snr_threshold.value} dB)')
                ax4.set_ylabel('SNR (dB)')
                ax4.set_title('Signal-to-Noise Ratio')
                ax4.legend()
            
            plt.tight_layout()
            plt.show()
        
        elif mode == 'gaps':
            fig, axes = plt.subplots(1, 2, figsize=(14, 6))
            
            # Gap visualization
            ax1 = axes[0]
            gap_map = np.isnan(signal).astype(float)
            im1 = ax1.imshow(gap_map[:, :, slice_idx], cmap='Reds', origin='lower', vmin=0, vmax=1)
            ax1.set_title('Gap Regions (Red = Missing)')
            ax1.set_xlabel('X')
            ax1.set_ylabel('Y')
            plt.colorbar(im1, ax=ax1, label='Gap Flag')
            
            # Gap statistics
            ax2 = axes[1]
            if 'completeness' in quality_metrics:
                comp = quality_metrics['completeness']
                coverage = comp.get('coverage', 0)
                gap_count = comp.get('gap_count', 0)
                
                ax2.bar(['Coverage', 'Gaps'], [coverage * 100, gap_count / 1000], 
                       color=['green', 'red'])
                ax2.set_ylabel('Percentage / Count (×1000)')
                ax2.set_title('Gap Statistics')
                ax2.tick_params(axis='x', rotation=45)
            
            plt.tight_layout()
            plt.show()
        
        else:  # trends
            fig, axes = plt.subplots(1, 2, figsize=(14, 6))
            
            # Temporal quality trend (simplified - using Z axis)
            ax1 = axes[0]
            if assessment_grid:
                # Calculate quality per layer (Z slice)
                quality_per_layer = []
                for z in range(signal.shape[2]):
                    layer_signal = signal[:, :, z]
                    layer_quality = 1.0 - (np.sum(np.isnan(layer_signal)) / layer_signal.size)
                    quality_per_layer.append(layer_quality)
                
                ax1.plot(quality_per_layer, marker='o')
                ax1.set_xlabel('Layer (Z)')
                ax1.set_ylabel('Quality Score')
                ax1.set_title('Quality Trend (Temporal)')
                ax1.grid(True, alpha=0.3)
            
            # Spatial quality trend (simplified - using X axis)
            ax2 = axes[1]
            if assessment_grid:
                # Calculate quality per X position
                quality_per_x = []
                for x in range(signal.shape[0]):
                    x_signal = signal[x, :, :]
                    x_quality = 1.0 - (np.sum(np.isnan(x_signal)) / x_signal.size)
                    quality_per_x.append(x_quality)
                
                ax2.plot(quality_per_x, marker='o', color='green')
                ax2.set_xlabel('X Position')
                ax2.set_ylabel('Quality Score')
                ax2.set_title('Quality Trend (Spatial)')
                ax2.grid(True, alpha=0.3)
            
            plt.tight_layout()
            plt.show()

# Connect events
def save_quality_assessment(button):
    """Save quality assessment results to MongoDB."""
    global quality_results, quality_metrics, current_grid_id, current_model_id, loaded_grid_data
    
    if not quality_results or not quality_metrics:
        warning_display.value = "<span style='color: red;'>⚠️ Please run assessment first</span>"
        return
    
    if not voxel_storage or not current_grid_id:
        warning_display.value = "<span style='color: red;'>⚠️ No grid loaded from MongoDB</span>"
        return
    
    status_display.value = "<b>Status:</b> Saving quality assessment..."
    progress_bar.value = 0
    warning_display.value = ""
    
    try:
        # Get grid document
        grid_doc = voxel_storage.mongo_client.db['voxel_grids'].find_one({'grid_id': current_grid_id})
        if not grid_doc:
            warning_display.value = "<span style='color: red;'>⚠️ Grid not found in MongoDB</span>"
            return
        
        # Prepare quality assessment metadata
        assessment_timestamp = datetime.now().isoformat()
        assessment_metadata = {
            'quality_assessment': {
                'assessment_timestamp': assessment_timestamp,
                'overall_score': quality_results.get('overall_score', 0.0),
                'assessment_type': assessment_type.value,
                'metrics': quality_metrics,
                'assessment_config': {
                    'completeness_threshold': completeness_threshold.value,
                    'snr_threshold': snr_threshold.value,
                    'uncertainty_threshold': uncertainty_threshold.value,
                    'confidence_threshold': confidence_threshold.value,
                    'max_error': max_error.value,
                    'temporal_tolerance': temporal_tolerance.value,
                    'spatial_tolerance': spatial_tolerance.value,
                    'gap_detection': gap_detection.value,
                    'gap_size_threshold': gap_size_threshold.value
                }
            }
        }
        
        # Update grid document with quality assessment
        metadata = grid_doc.get('metadata', {})
        if 'quality_assessments' not in metadata:
            metadata['quality_assessments'] = []
        
        # Add new assessment
        metadata['quality_assessments'].append(assessment_metadata['quality_assessment'])
        
        # Update latest quality assessment
        metadata['latest_quality_assessment'] = assessment_metadata['quality_assessment']
        
        # Update grid document
        voxel_storage.mongo_client.db['voxel_grids'].update_one(
            {'grid_id': current_grid_id},
            {'$set': {'metadata': metadata}}
        )
        
        progress_bar.value = 100
        status_display.value = f"<b>Status:</b> <span style='color: green;'>✅ Quality assessment saved to grid {current_grid_id[:8]}...</span>"
        warning_display.value = ""
        
    except Exception as e:
        warning_display.value = f"<span style='color: red;'>❌ Error saving assessment: {str(e)}</span>"
        status_display.value = f"<b>Status:</b> <span style='color: red;'>Error saving assessment</span>"
        progress_bar.value = 0
        import traceback
        traceback.print_exc()

execute_button.on_click(execute_assessment)
save_assessment_button.on_click(save_quality_assessment)
viz_mode.observe(lambda x: update_visualization(), names='value')
colormap_selector.observe(lambda x: update_visualization(), names='value')
quality_threshold_overlay.observe(lambda x: update_visualization(), names='value')

# ============================================
# Main Layout
# ============================================

main_layout = VBox([
    top_panel,
    HBox([left_panel, center_panel, right_panel]),
    bottom_panel
])

# Display the interface
display(main_layout)


## Summary

Congratulations! You've learned how to assess data quality in voxel grids.

### Key Takeaways

1. **Data Quality Metrics**: Completeness, coverage, consistency, accuracy, and reliability
2. **Signal Quality**: SNR, uncertainty, and confidence scores
3. **Alignment Accuracy**: Coordinate, temporal, and spatial accuracy validation
4. **Completeness Analysis**: Gap detection and filling strategies
5. **Quality Visualization**: Quality maps, metrics dashboards, and trend analysis

### Next Steps

Proceed to:
- **08_Quality_Dashboard.ipynb** - Learn to create interactive quality dashboards
- **09_Statistical_Analysis.ipynb** - Learn statistical analysis methods

### Related Resources

- Quality Module Documentation: `../docs/AM_QADF/05-modules/quality.md`
- API Reference: `../docs/AM_QADF/06-api-reference/quality-api.md`
- Examples: `../examples/`
